In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
import csv
import pandas
from keras.layers import Activation
from keras import backend
from keras.utils.generic_utils import get_custom_objects
import math

Using TensorFlow backend.


In [25]:
def compile_model (FIRST_LAYER_NEURONS, HIDDEN_LYAYER_NEURONS, INPUT, OUTPUT, ACTIVITION = "", CUSTOM_ACTIVITION =None , CUSTOM_ACTIVITION_NAME = ""):
    m = Sequential()
    if ACTIVITION == "" :
        m.add(Dense(FIRST_LAYER_NEURONS, input_dim=INPUT))
        #m.add(Dense(HIDDEN_LYAYER_NEURONS))
        m.add(Dense(OUTPUT))
        m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    elif ACTIVITION == "1":
        m.add(Activation(CUSTOM_ACTIVITION))
        m.add(Dense(FIRST_LAYER_NEURONS, input_dim=INPUT, activation=CUSTOM_ACTIVITION_NAME))
        #m.add(Dense(HIDDEN_LYAYER_NEURONS, activation=CUSTOM_ACTIVITION_NAME))
        m.add(Dense(OUTPUT, activation=CUSTOM_ACTIVITION_NAME))
        m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else :
        m.add(Dense(FIRST_LAYER_NEURONS, input_dim=INPUT, activation=ACTIVITION))
        #m.add(Dense(HIDDEN_LYAYER_NEURONS, activation=ACTIVITION))
        m.add(Dense(OUTPUT, activation=ACTIVITION))
        m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return m

In [26]:
def set_model (FIRST_LAYER_NEURONS, HIDDEN_LYAYER_NEURONS, TRAINING_PARAMETERS, TRAINGING_RESULTS, TESTING_PARAMETERS, TESTING_RESULTS, EPOCHS, ACTIVITION = "", CUSTOM_ACTIVITION =None , CUSTOM_ACTIVITION_NAME = ""):
    # create model
    model = compile_model(FIRST_LAYER_NEURONS,HIDDEN_LYAYER_NEURONS,TRAINING_PARAMETERS.shape[1],TRAINGING_RESULTS.shape[1],ACTIVITION, CUSTOM_ACTIVITION,CUSTOM_ACTIVITION_NAME )
    # Fit the model
    model.fit(TRAINING_PARAMETERS, TRAINGING_RESULTS, epochs=EPOCHS, batch_size=10)
    # evaluate the model
    scores = model.evaluate(TESTING_PARAMETERS, TESTING_RESULTS)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return model

In [128]:
def datasets_predict(PARAMETERS, RESULTS,THE_SEPERATOR_ROW):
    model = set_model(PARAMETERS.shape[1],
                      round((PARAMETERS.shape[1]+RESULTS.shape[1]*2)/3),
                      PARAMETERS[:THE_SEPERATOR_ROW],
                      RESULTS[:THE_SEPERATOR_ROW],
                      PARAMETERS[THE_SEPERATOR_ROW:],
                      RESULTS[THE_SEPERATOR_ROW:], 
                      19,
                      "relu")
    return model.predict(PARAMETERS[THE_SEPERATOR_ROW:]), RESULTS[THE_SEPERATOR_ROW:]

In [131]:
Dataset = pandas.read_excel("Fuzzyfier3.xlsx", sheet_name='Fuzzyfied')
Parameters = pandas.DataFrame(Dataset, columns = ['ModerateWind', 'ModerateHumidity', 'ModerateRain', 'ModeratePreassure'])
Results= pandas.DataFrame(Dataset, columns = ['RoundedTemp'])

In [132]:
predection, right_answers = datasets_predict(Parameters.values,Results.values,3226)

Epoch 1/19
3226/3226 [==============================] - 3s 816us/step - loss: 2.0703 - acc: 0.3484
Epoch 2/19
3226/3226 [==============================] - 1s 330us/step - loss: 1.0203 - acc: 0.2046
Epoch 3/19
3226/3226 [==============================] - 1s 327us/step - loss: 0.9611 - acc: 0.1826
Epoch 4/19
3226/3226 [==============================] - 1s 327us/step - loss: 0.9247 - acc: 0.1885
Epoch 5/19
3226/3226 [==============================] - 1s 326us/step - loss: 0.8891 - acc: 0.1739
Epoch 6/19
3226/3226 [==============================] - 1s 334us/step - loss: 0.8503 - acc: 0.1875
Epoch 7/19
3226/3226 [==============================] - 1s 346us/step - loss: 0.8066 - acc: 0.1844
Epoch 8/19
3226/3226 [==============================] - 1s 418us/step - loss: 0.7610 - acc: 0.2353
Epoch 9/19
3226/3226 [==============================] - 1s 331us/step - loss: 0.7099 - acc: 0.4436
Epoch 10/19
3226/3226 [==============================] - 1s 321us/step - loss: 0.6518 - acc: 0.6891
Epoch 11/

In [133]:
Results_rights= pandas.DataFrame(Dataset, columns = ['ModerateTemp'])
m =[[defuzzyfy(correct_rate(a[0]), 19.67, 21.33),defuzzyfy(b[0],19.67,21.33)] for a,b in zip(predection, Results_rights.values)]
m

[[26.044894321556335, 26.702234375],
 [24.71643310781051, 26.435609375],
 [26.39379654397257, 25.9356875],
 [27.74605550350467, 25.06915625],
 [28.31824903418172, 24.535906249999996],
 [28.277995440548978, 24.569234375],
 [28.357628924528093, 24.769203125000004],
 [28.49112306089172, 24.502578125],
 [28.480181836624208, 24.502578125],
 [27.793318890517018, 23.936],
 [27.068556035155844, 23.5360625],
 [28.56549219380013, 22.569546874999997],
 [30.448613755427537, 22.33625],
 [30.335031217199457, 21.902984375000003],
 [29.812757216253402, 21.569703124999997],
 [29.08273920386815, 21.736343750000003],
 [29.72521103702179, 22.902828125000003],
 [29.209626050676356, 24.435921875],
 [28.3916703500673, 25.635734375000002],
 [26.966044701964464, 26.202312499999998],
 [27.387940242924536, 26.00234375],
 [28.521613340628676, 24.202624999999998],
 [29.44447550768368, 22.902828124999996],
 [30.00384471415476, 21.703015625],
 [29.461823993736708, 21.902984375000003],
 [29.1864504826014, 21.26975],


In [120]:
def correct_rate(x):
    return 0.7*(math.asin(math.sqrt(x/2)))**0.6

In [111]:
correct_rate(2)

0.9424777960769379

In [104]:
def defuzzyfy(x,min,range):
    return x*range+min

In [161]:
def fuzzyfy_list(x):
    refined = refine(x)
    max_v = max(refined)
    min_v = min(refined)
    outlier_min , oulier_max = outiler_limits(refined)
    lowest = max(max_v, oulier_min)
    hiest = min(min_v, oulier_max)
    division_range = hiest-lowest
    fuzzyfied = [min(1,max(0,(p-lowest)/division_range))]
    return fuzzyfied,refined,lowest,division_range

In [159]:
def refine(l):
    m = [0]*len(l)
    m[0] = l[0]
    for i in range(1,len(l)-1):
        m[i] = (l[i-1]+l[i]+l[i+1])/3
    m[len(l)-1] = l[len(l)-1]
    return m


In [155]:
def quartile(x, n = 1):
    s = x
    s.sort()
    print(s)
    i = round(len(s)*n/4,0)
    i = int(i)
    return x[i]

In [157]:
outlier_limits([1,2,3,85,7,9])

[1, 2, 3, 7, 9, 85]
[1, 2, 3, 7, 9, 85]


(-6.0, 18.0)

In [148]:
def outlier_limits(x):
    q1 = quartile(x)
    q3 = quartile(x,3)
    qrange = q3 - q1
    tolerance = qrange * 1.5
    low = q1 - tolerance
    hi = q3 + tolerance
    return low, hi